# Лабораторная работа: Бинарные деревья (BST, обходы, балансировка)

## Цели работы
1. Закрепить базовые определения: узел, степень узла, высота/глубина, листья, полнота/сбалансированность.
2. Реализовать структуры и алгоритмы: узел дерева, вставка/поиск/удаление в BST, обходы DFS (LNR/ NLR/ LRN) и BFS.
3. Научиться оценивать сложность операций и экспериментально измерять их на наборах данных.
4. Научиться проверять свойства дерева: высота, сбалансированность (по критерию |h(left) - h(right)| ≤ 1), корректность BST-инварианта.
5. Освоить простую визуализацию структуры дерева в текстовом виде.

## Краткая теория
**Бинарное дерево** — иерархическая структура данных, где у каждого узла не более двух потомков (левый и правый). **Двоичное дерево поиска (BST)** поддерживает инвариант: для любого узла `x` все элементы в левом поддереве меньше `x.key`, а в правом — больше. Классические обходы: `inorder (LNR)`, `preorder (NLR)`, `postorder (LRN)`; обход в ширину — `BFS` с очередью. Амортизированная сложность для случайных вставок в BST близка к `O(log n)`, в худшем случае (вырожденное дерево) — `O(n)`.

В качестве опорного материала используйте предоставленную лекцию по бинарным деревьям. См. конспект/слайды.




In [1]:
# Базовые импорты (старайтесь не добавлять внешние библиотеки)
from collections import deque
import random, math, time
from typing import Optional, List, Tuple, Any

class Node:
    __slots__ = ("key", "left", "right")
    def __init__(self, key: int, left: 'Optional[Node]'=None, right: 'Optional[Node]'=None):
        self.key = key
        self.left = left
        self.right = right

    def __repr__(self):
        return f"Node({self.key})"


### Задача 1. Конструктор BST и обходы
Реализуйте:
1. `bst_insert(root, key)` — вставка ключа в BST.
2. `bst_search(root, key)` — поиск ключа.
3. Обходы: `inorder`, `preorder`, `postorder`, `bfs`.
4. Сервис: `build_bst(iterable)` — постройте BST последовательными вставками.


In [3]:
def bst_insert(root: Optional[Node], key: int) -> Node:
    if root is None:
        return Node(key)
    cur = root
    while True:
        if key < cur.key:
            if cur.left is None:
                cur.left = Node(key)
                break
            cur = cur.left
        elif key > cur.key:
            if cur.right is None:
                cur.right = Node(key)
                break
            cur = cur.right
        else:
            # уже существует — ничего не делаем
            break
    return root

def bst_search(root: Optional[Node], key: int) -> Optional[Node]:
    cur = root
    while cur:
        if key < cur.key:
            cur = cur.left
        elif key > cur.key:
            cur = cur.right
        else:
            return cur
    return None

def inorder(root: Optional[Node]) -> List[int]:
    res = []
    def dfs(node):
        if not node:
            return
        dfs(node.left)
        res.append(node.key)
        dfs(node.right)

    dfs(root)
    return res

def preorder(root: Optional[Node]) -> List[int]:
    res = []
    def dfs(node):
        if not node:
            return
        res.append(node.key)
        dfs(node.left)
        dfs(node.right)
    dfs(root)
    return res

def postorder(root: Optional[Node]) -> List[int]:
    res = []
    def dfs(node):
        if not node:
            return
        dfs(node.left)
        dfs(node.right)
        res.append(node.key)
    dfs(root)
    return res

def bfs(root: Optional[Node]) -> List[int]:
    if not root:
        return []
    q, res = deque([root]), []
    while q:
        v = q.popleft()
        res.append(v.key)
        if v.left: q.append(v.left)
        if v.right: q.append(v.right)
    return res

def build_bst(iterable) -> Optional[Node]:
    root = None
    for x in iterable:
        root = bst_insert(root, x)
    return root


In [4]:
# Автотесты к Задаче 1
data = [7, 3, 9, 1, 5, 8, 10]
root = build_bst(data)
assert inorder(root) == [1,3,5,7,8,9,10], "inorder неверен"
assert preorder(root) == [7,3,1,5,9,8,10], "preorder неверен"
assert postorder(root) == [1,5,3,8,10,9,7], "postorder неверен"
assert bfs(root) == [7,3,9,1,5,8,10], "bfs неверен"
assert bst_search(root, 8) and bst_search(root, 8).key == 8, "search не находит 8"
assert bst_search(root, 2) is None, "search должен вернуть None для отсутствующих ключей"
print("Задача 1 — OK")


Задача 1 — OK


### Задача 2. Высота, сбалансированность и инвариант BST
Реализуйте:
- `height(root)` — высота дерева (кол-во рёбер на самом длинном пути; для пустого — -1).
- `is_balanced(root)` — сбалансировано ли дерево по правилу |h(L) - h(R)| ≤ 1 для каждого узла.
- `is_valid_bst(root)` — проверка инварианта BST (строгое неравенство).


In [5]:
def height(root: Optional[Node]) -> int:
    if root is None:
        return -1
    return 1 + max(height(root.left), height(root.right))

def is_balanced(root: Optional[Node]) -> bool:
    ok = True
    def dfs(node) -> int:
        nonlocal ok
        if not node:
            return -1
        hl = dfs(node.left)
        hr = dfs(node.right)
        if abs(hl - hr) > 1:
            ok = False
        return 1 + max(hl, hr)
    dfs(root)
    return ok

def is_valid_bst(root: Optional[Node]) -> bool:
    def check(node, lo, hi) -> bool:
        if not node:
            return True
        if not (lo < node.key < hi):
            return False
        return check(node.left, lo, node.key) and check(node.right, node.key, hi)
    return check(root, float("-inf"), float("inf"))


In [6]:
# Автотесты к Задаче 2
root = build_bst([4,2,6,1,3,5,7])
assert height(root) == 2
assert is_balanced(root) is True
assert is_valid_bst(root) is True

# создадим несбалансированное дерево
root2 = None
for x in range(10):
    root2 = bst_insert(root2, x)
assert is_balanced(root2) is False
assert is_valid_bst(root2) is True
print("Задача 2 — OK")


Задача 2 — OK


### Задача 3. Удаление в BST, k-й по порядку элемент, LCA
Реализуйте:
- `bst_delete(root, key)` — удаление узла из BST (3 случая: лист, один ребёнок, два ребёнка).
- `kth_smallest(root, k)` — k-й статистический (0-индексация по возрастанию).
- `lca_bst(root, a, b)` — Lowest Common Ancestor в BST.


In [22]:
def bst_delete(root: Optional[Node], key: int) -> Optional[Node]:
    if root is None:
        return None
    if key < root.key:
        root.left = bst_delete(root.left, key)
    elif key > root.key:
        root.right = bst_delete(root.right, key)
    else:
        # нашли
        if root.left is None:
            return root.right
        if root.right is None:
            return root.left
        # два ребёнка: ищем min в правом поддереве
        cur = root.right
        while cur.left:
            cur = cur.left
        root.key = cur.key
        root.right = bst_delete(root.right, cur.key)
    return root

def kth_smallest(root: Optional[Node], k: int) -> int:
    # inorder-порядок
    stack, cur = [], root
    count = 0
    while stack or cur:
        while cur:
            stack.append(cur)
            cur = cur.left
        cur = stack.pop()
        if count == k:
            return cur.key
        count += 1
        cur = cur.right
    raise IndexError("k is out of range")


def lca_bst(root: Optional[Node], a: int, b: int) -> Optional[Node]:
    cur = root
    lo, hi = min(a,b), max(a,b)
    while cur:
        if hi < cur.key:
            cur = cur.left
        elif lo > cur.key:
            cur = cur.right
        else:
            return cur
    return None


In [24]:
# Автотесты к Задаче 3
root = build_bst([20,10,30,5,15,25,40,13,17])
assert inorder(root) == sorted([20,10,30,5,15,25,40,13,17])
root = bst_delete(root, 10)
assert inorder(root) == [5,13,15,17,20,25,30,40]
assert kth_smallest(root, 0) == 5
assert kth_smallest(root, 3) == 17
assert lca_bst(root, 5, 17).key == 13
print("Задача 3 — OK")


Задача 3 — OK


### Задача 4. Построение по уровневому представлению и ASCII-визуализация
1. Реализуйте `build_tree_level(lst)`, где `lst` — список значений уровнями (используйте `None` для пустых узлов).
2. Реализуйте `ascii_tree(root)` — вывод дерева в виде моноширинного текста.


In [46]:
def build_tree_level(lst: List[Optional[int]]) -> Optional[Node]:
    if not lst:
        return None

    it = iter(lst + [None])
    root_val = next(it)

    if root_val is None:
        return None

    root = Node(root_val)
    q = deque([root])

    for a,b in zip(it, it):
        cur = q.popleft()

        if a is not None:
            cur.left = Node(a)
            q.append(cur.left)

        if b is not None:
            cur.right = Node(b)
            q.append(cur.right)
    return root

def ascii_tree(root: Optional[Node]) -> str:
    if not root:
        return "<empty>"
    # Вернём список строк для каждого поддерева
    def _build_ascii_block(node: Optional[Node]) -> tuple[List[str], int, int, int]:
        """
        Рекурсивно строит ASCII-блок для поддерева.

        :param node: текущий узел
        :return: кортеж:
            - lines: список строк с ASCII-представлением поддерева
            - total_width: общая ширина блока в символах
            - total_height: количество строк в блоке
            - root_position: горизонтальная позиция центра ключа относительно начала блока
        """
        if node is None:
            return [], 0, 0, 0

        key_str = str(node.key)
        key_width = len(key_str)

        # Листовой узел — без потомков
        if not node.left and not node.right:
            return [key_str], key_width, 1, key_width // 2

        # Строим ASCII-блоки для левого и правого поддеревьев
        left_lines, left_width, left_height, left_root_pos = _build_ascii_block(node.left)
        right_lines, right_width, right_height, right_root_pos = _build_ascii_block(node.right)

        # Выравниваем высоты поддеревьев
        if left_height < right_height:
            left_lines += [" " * left_width] * (right_height - left_height)
            left_height = right_height
        elif right_height < left_height:
            right_lines += [" " * right_width] * (left_height - right_height)
            right_height = left_height

        # Строка с ключом узла (корня поддерева)
        root_line = (" " * left_width) + key_str + (" " * right_width)

        # Строка со стрелками к потомкам (если потомки существуют)
        left_branch = (" " * left_root_pos + "/" + " " * (left_width - left_root_pos - 1)) if left_width > 0 else " " * left_width
        right_branch = (" " * right_root_pos + "\\" + " " * (right_width - right_root_pos - 1)) if right_width > 0 else " " * right_width
        branches_line = left_branch + (" " * key_width) + right_branch

        # Объединяем строки левого и правого поддеревьев построчно
        merged_subtrees = [
            left_line + (" " * key_width) + right_line
            for left_line, right_line in zip(
                left_lines if left_lines else [" " * left_width] * left_height,
                right_lines if right_lines else [" " * right_width] * right_height
            )
        ]

        # Итоговые строки для текущего поддерева
        all_lines = [root_line, branches_line] + merged_subtrees
        total_width = left_width + key_width + right_width
        total_height = left_height + 2
        root_position = left_width + key_width // 2

        return all_lines, total_width, total_height, root_position

    # Строим блок для всего дерева и склеиваем строки
    ascii_lines, *_ = _build_ascii_block(root)
    return "\n".join(ascii_lines)


In [47]:
# Автотесты к Задаче 4
root = build_tree_level([7,3,9,1,5,8,10])
print(ascii_tree(root))
print("Задача 4 — визуализация напечатана (проверьте структурно).")


   7    
 /   \  
 3   9  
/ \ /  \
1 5 8 10
Задача 4 — визуализация напечатана (проверьте структурно).


## Контрольные вопросы
1. Чем различаются `inorder`, `preorder`, `postorder`? Для какого обхода BST даёт отсортированную последовательность ключей и почему?
2. Приведите пример последовательности вставок, при которой высота BST равна `n-1`.
3. Почему в худшем случае сложность поиска/вставки в BST равна `O(n)`? Как этого избежать?
4. В чём идея балансировки (напр., AVL/Red-Black)? Какой инвариант они поддерживают поверх инварианта BST?
5. Какие преимущества и недостатки у представления дерева в виде связанной структуры узлов
